### Orbis and PPP Loan Data Merge

1. Extract matches between the PPP Loan Data and Orbis
2. Get all businesses NOT in Orbis but is in Bay Area AND is Minority-owned
3. Get needed columns from PPP Loan to append to Orbis
4. Extract PPP Loan columns and merge with orbis
5. Add PPP bus not in Orbis with those PPP loan columns
6. Reexport and update everyone

In [1]:
from cleanco import cleanco
import pandas as pd
import re
import os
from os.path import dirname, abspath
from IPython.display import display, HTML # Source: https://stackoverflow.com/questions/38783027/jupyter-notebook-display-two-pandas-tables-side-by-side

In [2]:
# Get Current Working Directory and Parent Path (for reading files in different folders)
## Source: https://stackoverflow.com/questions/30218802/get-parent-of-current-directory-from-python-script/30218825
d = dirname(dirname(abspath(os.getcwd())))
d

'C:\\Users\\Trang\\Desktop\\cp-research\\afn'

In [ ]:
orbis_file = d+'/ppp-loan-data/Orbis database_six cities only.csv'
ppp_file =d+'/ppp-loan-data/out/bay_bus_from_ppp.csv'

In [ ]:
orbis_df = pd.read_csv(orbis_file)
orbis_df.head()

In [ ]:
ppp_df = pd.read_csv(ppp_file)
ppp_df.head()

In [ ]:
# Clean Orbis Name Column
orbis_df['Company Name - Clean'] =orbis_df['Company name Latin alphabet'].apply(lambda x: cleanco(x).clean_name())
orbis_df.head()

In [ ]:
ppp_df['BorrowerName - Clean'] = ppp_df['BorrowerName'].apply(str).apply(lambda x: cleanco(x).clean_name())
ppp_df.head()

In [ ]:
# Find businesses in Orbis databases
ppp_from_orbis = ppp_df[ppp_df['BorrowerName - Clean'].isin(orbis_df['Company Name - Clean'])]
ppp_from_orbis.head()

In [ ]:
ppp_columns = ['BorrowerName - Clean','CurrentApprovalAmount', 'YearApproved', 'Race', 'Ethnicity', 'Minority', 'NAICS_4','IndustrySubsector']

### Get Businesses Not in Orbis

In [ ]:
ppp_not_in_orbis = ppp_df[~ppp_df['BorrowerName - Clean'].isin(orbis_df['Company Name - Clean'])]
ppp_not_in_orbis.head()

In [ ]:
ppp_not_in_orbis.columns

In [ ]:
ppp_to_orbis_columns = ['BorrowerName','BorrowerAddress','BorrowerCity', 'BorrowerState', 'BorrowerZip','Gender', 'ProjectCity', 'BusinessType']

In [ ]:
def extract_columns(df, columns):
    return df[columns]

In [ ]:
extracted_ppp = extract_columns(ppp_not_in_orbis, ppp_to_orbis_columns + ppp_columns)
extracted_ppp.head()

In [ ]:
extracted_ppp_minority = extracted_ppp[extracted_ppp['Minority'] == 'Yes']
extracted_ppp_minority.head()

In [ ]:
for i in extracted_ppp_minority['BorrowerCity'].str.title().unique():
    print(i)

In [ ]:
extracted_ppp_minority.to_csv(d+'/ppp-loan-data/out/ppp-not-in-orbis-minority.csv', index=False)

In [ ]:
# Read in New combined files

revised_orbis_ppp_df = pd.read_csv(d+'/ppp-loan-data/out/merged_cities/orbis_ppp_merge.csv')
revised_orbis_ppp_df.head()

In [ ]:
revised_orbis_ppp_df.columns

In [ ]:
revised_orbis_ppp_df['Address line 1\nLatin Alphabet'] = revised_orbis_ppp_df['Address line 1\nLatin Alphabet'].str.upper()
revised_orbis_ppp_df['City\nLatin Alphabet'] = revised_orbis_ppp_df['City\nLatin Alphabet'].str.upper()
revised_orbis_ppp_df['Study Area'] = revised_orbis_ppp_df['Study Area'].str.upper()

In [ ]:
revised_orbis_ppp_df.head()

In [ ]:
revised_orbis_ppp_df['Study Area'].unique()

In [ ]:
revised_orbis_ppp_df.to_csv(d+'/ppp-loan-data/out/orbis-ppp-merge-cleaned.csv', index=False)

In [ ]:
short_orbis_ppp_df = revised_orbis_ppp_df[revised_orbis_ppp_df['Study Area'].notna()]

In [ ]:
short_orbis_ppp_df.head()

In [ ]:
short_orbis_ppp_df.to_csv(d+'/ppp-loan-data/out/orbis-ppp-merge-study-areas.csv', index=False)

Please add in a new column in your spreadsheet called "Operational status"
If the business has an active license based on the business license data from your city list it as 'Active'
If the business has an expired license, list as 'Expired license'
If the business is closed, list as 'Closed'
If the business does not appear in the business license database at all note this as 'Not in license database'

In [3]:
naics_df = pd.read_csv(d+'/ppp-loan-data/naics/naics_4_study.csv')
naics_df.head()

,Sector,Subsector,Digit NAIC,Industry Group,Include in our study?
0,Retail Trade,Furniture & Home Furnishing Stores,4421,Furniture Stores,Yes
1,Retail Trade,Furniture & Home Furnishing Stores,4422,Home Furnishings Stores,Yes
2,Retail Trade,Clothing & Clothing Accessories Stores,4481,Clothing Stores,Yes
3,Retail Trade,Clothing & Clothing Accessories Stores,4482,Shoe Stores,Yes
4,Retail Trade,Clothing & Clothing Accessories Stores,4483,"Jewelry, Luggage, and Leather Goods Stores",Yes


In [4]:
naics_df.columns

Index(['Sector', 'Subsector', 'Digit NAIC', 'Industry Group',
       'Include in our study?'],
      dtype='object')

In [5]:
orbis_ppp_df_naics = pd.read_csv(d+'/ppp-loan-data/out/orbis-ppp-merge-study-areas.csv')
orbis_ppp_df_naics.head()

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Company name Latin alphabet,Inactive,Quoted,Branch,OwnData,Woco,Country ISO code,"NACE Rev. 2, core code (4 digits)",Consolidation code,Last avail. year,...,Study Area,Company Name - Clean,BorrowerName-Clean,CurrentApprovalAmount,YearApproved,Race,Ethnicity,Minority,NAICS_4,IndustrySubsector
0,CALIFORNIA NURSES ASSOCIATION,No,No,Yes,No,No,US,9412.0,LF,2019.0,...,OAKLAND,CALIFORNIA NURSES ASSOCIATION,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,HONDA CARS OF CORONA,No,No,No,No,No,US,4519.0,LF,2019.0,...,OAKLAND,HONDA CARS OF CORONA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,SIMI MANAGEMENT CORP,No,No,No,No,No,US,4519.0,LF,2019.0,...,OAKLAND,SIMI MANAGEMENT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ECONOMY LUMBER CO OF OAKLAND,No,No,No,No,No,US,4778.0,LF,2019.0,...,OAKLAND,ECONOMY LUMBER CO OF OAKLAND,ECONOMY LUMBER CO OF OAKLAND,807700.0,2020.0,Unanswered,Unknown/NotStated,Unanswered,4233.0,Lumber and Other Construction Materials Mercha...
4,BROADWAY FORD,No,No,No,No,No,US,4519.0,LF,2008.0,...,OAKLAND,BROADWAY FORD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
len(orbis_ppp_df_naics)

112682

In [7]:
orbis_ppp_df_naics.columns

Index(['Company name Latin alphabet', 'Inactive', 'Quoted', 'Branch',
       'OwnData', 'Woco', 'Country ISO code',
       'NACE Rev. 2, core code (4 digits)', 'Consolidation code',
       'Last avail. year',
       'Operating revenue (Turnover)\nth USD Last avail. yr',
       'Number of employees\nLast avail. yr', 'Address line 1\nLatin Alphabet',
       'Address line 2\nLatin Alphabet', 'Postcode\nLatin Alphabet',
       'City\nLatin Alphabet', 'Region in country', 'Latitude', 'Longitude',
       'Website address', 'NAICS 2017, core code (4 digits)',
       'NAICS 2017, core code - description', 'Size classification',
       'Type of entity', 'Ethnic minority owned indicator (in US)',
       'Woman owned indicator (in US)', 'Study Area', 'Company Name - Clean',
       'BorrowerName-Clean', 'CurrentApprovalAmount', 'YearApproved', 'Race',
       'Ethnicity', 'Minority', 'NAICS_4', 'IndustrySubsector'],
      dtype='object')

In [8]:
orbis_ppp_df_naics['NAICS 2017, core code (4 digits)']

0         8139.0
1         4411.0
2         4411.0
3         4441.0
4         4411.0
5         4411.0
6         4471.0
7         4422.0
8         4422.0
9         4471.0
10        4451.0
11        4539.0
12        4411.0
13        4451.0
14        4542.0
15        4422.0
16        4411.0
17        4412.0
18        4451.0
19        4461.0
20        4451.0
21        4431.0
22        4533.0
23        4533.0
24        4441.0
25        4441.0
26        4451.0
27        4471.0
28        4471.0
29        4461.0
           ...  
112652       NaN
112653       NaN
112654       NaN
112655       NaN
112656       NaN
112657       NaN
112658       NaN
112659       NaN
112660       NaN
112661       NaN
112662       NaN
112663       NaN
112664       NaN
112665       NaN
112666       NaN
112667       NaN
112668       NaN
112669       NaN
112670       NaN
112671       NaN
112672       NaN
112673       NaN
112674       NaN
112675       NaN
112676       NaN
112677       NaN
112678       NaN
112679       N

In [9]:
orbis_ppp_df_naics['NAICS_4']

0            NaN
1            NaN
2            NaN
3         4233.0
4            NaN
5            NaN
6            NaN
7         4441.0
8         4422.0
9            NaN
10           NaN
11           NaN
12           NaN
13           NaN
14           NaN
15        4422.0
16           NaN
17           NaN
18           NaN
19           NaN
20           NaN
21           NaN
22        4421.0
23        4421.0
24           NaN
25           NaN
26           NaN
27           NaN
28           NaN
29           NaN
           ...  
112652    5413.0
112653    5413.0
112654    7225.0
112655    8111.0
112656    4461.0
112657    4461.0
112658    7225.0
112659    2361.0
112660    7225.0
112661    4239.0
112662    5419.0
112663    6117.0
112664    5419.0
112665    5413.0
112666    8111.0
112667    5417.0
112668    7225.0
112669    6212.0
112670    6212.0
112671    5419.0
112672    7225.0
112673    6232.0
112674    7225.0
112675    7225.0
112676    5418.0
112677    5415.0
112678    2382.0
112679    5412

In [10]:
condition_1 = orbis_ppp_df_naics['NAICS_4'].isin(naics_df['Digit NAIC'])
condition_2 = orbis_ppp_df_naics['NAICS 2017, core code (4 digits)'].isin(naics_df['Digit NAIC']) 
orbis_ppp_df_naics = orbis_ppp_df_naics.loc[condition_1 | condition_2]
orbis_ppp_df_naics.head()

,Company name Latin alphabet,Inactive,Quoted,Branch,OwnData,Woco,Country ISO code,"NACE Rev. 2, core code (4 digits)",Consolidation code,Last avail. year,...,Study Area,Company Name - Clean,BorrowerName-Clean,CurrentApprovalAmount,YearApproved,Race,Ethnicity,Minority,NAICS_4,IndustrySubsector
7,DICKS WHOLESALE CARPET WAREHOUSE INC,No,No,No,No,No,US,4778.0,LF,2019.0,...,OAKLAND,DICKS WHOLESALE CARPET WAREHOUSE,DICKS WHOLESALE CARPET WAREHOUSE,173328.2,2020.0,Unanswered,Unknown/NotStated,Unanswered,4441.0,Building Material and Supplies Dealers
8,DICKS WHOLESALE CARPET WAREHOUSE INC,No,No,No,No,No,US,4778.0,LF,2019.0,...,OAKLAND,DICKS WHOLESALE CARPET WAREHOUSE,DICKS WHOLESALE CARPET WAREHOUSE,169656.0,2021.0,Unanswered,Unknown/NotStated,Unanswered,4422.0,Home Furnishings Stores
11,DIGICOM,No,No,No,No,No,US,4778.0,LF,2019.0,...,OAKLAND,DIGICOM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15,BAY AREA CONTRACT CARPETS INC,No,No,No,No,No,US,4778.0,LF,2019.0,...,OAKLAND,BAY AREA CONTRACT CARPETS,BAY AREA CONTRACT CARPETS,74219.0,2020.0,Unanswered,Unknown/NotStated,Unanswered,4422.0,Home Furnishings Stores
22,FRIANT & ASSOCIATES INC,No,No,No,No,No,US,4779.0,LF,2019.0,...,OAKLAND,FRIANT & ASSOCIATES,FRIANT & ASSOCIATES,2842000.0,2020.0,Unanswered,Unknown/NotStated,Unanswered,4421.0,Furniture Stores


In [18]:
len(orbis_ppp_df_naics)

56502

In [23]:
orbis_ppp_df_naics['NAICS 2017, core code (4 digits)'].unique()

array([4422., 4539., 4533., 4511., 4512., 4453., 4421., 4483., 4532.,
       4522., 4481., 4451., 4523., 4482., 4531., 4541., 4452., 4441.,
       4431., 4543., 4412., 4542., 4471., 4461., 4411., 7223., 8129.,
       8123., 7224., 7211., 7225., 8121., 8134., 8113., 8111., 8131.,
       8132., 8133., 4884., 8139., 4413., 8112., 8114., 4442.,   nan])

In [24]:
orbis_ppp_df_naics.to_csv(d+'/ppp-loan-data/out/orbis-ppp-merge-naics.csv', index=False)